**Keras MNIST PQT/QAT Tutorial**

Load tensorflow library

In [2]:
import os 
import tensorflow as tf
print(tf.__version__)

!pip install tensorflow_model_optimization

2.9.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/Colab Embedded2022'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load dataset (MNIST)

In [4]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = y_train.flatten()
y_test = y_test.flatten()


Preprocessing dataset

In [5]:
import numpy as np
input_shape = (32, 32, 3)

x_train = x_train.reshape(x_train.shape[0], *input_shape)
x_train = x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], *input_shape)
x_test = x_test / 255.0

y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)


Define and train the baseline model 

In [6]:
batch_size = 64
epochs = 50


reg = tf.keras.regularizers.l2(1e-4)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', input_shape=input_shape, kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(16, 3, padding='same', kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),    

    tf.keras.layers.Conv2D(32, 3, padding='same', kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(32, 3, padding='same', kernel_regularizer=reg),    
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, 3, padding='same', kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(64, 3, padding='same', kernel_regularizer=reg),    
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax', kernel_regularizer=reg)
])

schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-3, decay_steps=x_train.shape[0] * epochs // batch_size)


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=schedule, decay=0),
            loss='categorical_crossentropy', metrics=['acc'])


from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )

#history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch = len(x_train) / batch_size, epochs=epochs, validation_data=(x_test, y_test))

Epoch 1/50
781/781 [==============================] - 43s 40ms/step - loss: 1.4161 - acc: 0.4962 - val_loss: 1.3339 - val_acc: 0.5223
Epoch 2/50
781/781 [==============================] - 29s 37ms/step - loss: 1.1057 - acc: 0.6152 - val_loss: 1.6758 - val_acc: 0.4857
Epoch 3/50
781/781 [==============================] - 29s 37ms/step - loss: 1.0046 - acc: 0.6543 - val_loss: 1.6221 - val_acc: 0.5192
Epoch 4/50
781/781 [==============================] - 30s 39ms/step - loss: 0.9333 - acc: 0.6829 - val_loss: 1.1327 - val_acc: 0.6291
Epoch 5/50
781/781 [==============================] - 30s 38ms/step - loss: 0.8765 - acc: 0.7051 - val_loss: 1.0186 - val_acc: 0.6566
Epoch 6/50
781/781 [==============================] - 30s 39ms/step - loss: 0.8342 - acc: 0.7204 - val_loss: 1.2728 - val_acc: 0.6243
Epoch 7/50
781/781 [==============================] - 30s 38ms/step - loss: 0.7995 - acc: 0.7358 - val_loss: 0.9025 - val_acc: 0.7085
Epoch 8/50
781/781 [==============================] - 30s 38ms

In [7]:
tf.keras.models.save_model(model, os.path.join(base_path, "model.h5"))
model.save_weights(os.path.join(base_path, "model.weight"))

In [8]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model) # path to the SavedModel directory
tflite_model = converter.convert()
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Save the model.
with open(os.path.join(base_path, 'model.tflite'), 'wb') as f:
  f.write(tflite_model)

Apply Quantization-aware Training

In [9]:
from tensorflow_model_optimization.quantization.keras import quantize_apply
from tensorflow_model_optimization.quantization.keras import quantize_annotate_layer

batch_size = 64
epochs = 30

reg = tf.keras.regularizers.l2(1e-4)

model = tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(16, 3, padding='same', input_shape=input_shape, kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(16, 3, padding='same', kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),    

    tf.keras.layers.Conv2D(32, 3, padding='same', kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(32, 3, padding='same', kernel_regularizer=reg),    
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(64, 3, padding='same', kernel_regularizer=reg),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    quantize_annotate_layer(tf.keras.layers.Conv2D(64, 3, padding='same', kernel_regularizer=reg)),    
    quantize_annotate_layer(tf.keras.layers.BatchNormalization()),
    quantize_annotate_layer(tf.keras.layers.ReLU()),
    
    quantize_annotate_layer(tf.keras.layers.GlobalAveragePooling2D()),
    quantize_annotate_layer(tf.keras.layers.Flatten()),
    quantize_annotate_layer(tf.keras.layers.Dense(10, activation='softmax', kernel_regularizer=reg))
   ])

model.load_weights(os.path.join(base_path, "model.weight"))

schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=1e-3, decay_steps=x_train.shape[0] * epochs // batch_size)



from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
    #zoom_range=0.3
    )



quantized_model = quantize_apply(model)
quantized_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=schedule, decay=0),
            loss='categorical_crossentropy', metrics=['acc'])

quantized_model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                   steps_per_epoch = len(x_train) / batch_size, epochs=epochs, validation_data=(x_test, y_test))


Epoch 1/30
781/781 [==============================] - 33s 40ms/step - loss: 0.8138 - acc: 0.7516 - val_loss: 0.7769 - val_acc: 0.7571
Epoch 2/30
781/781 [==============================] - 31s 40ms/step - loss: 0.6472 - acc: 0.7957 - val_loss: 1.0035 - val_acc: 0.7135
Epoch 3/30
781/781 [==============================] - 31s 39ms/step - loss: 0.6299 - acc: 0.8024 - val_loss: 0.8382 - val_acc: 0.7484
Epoch 4/30
781/781 [==============================] - 31s 40ms/step - loss: 0.6154 - acc: 0.8070 - val_loss: 0.6822 - val_acc: 0.7917
Epoch 5/30
781/781 [==============================] - 31s 40ms/step - loss: 0.6019 - acc: 0.8124 - val_loss: 0.6355 - val_acc: 0.8045
Epoch 6/30
781/781 [==============================] - 34s 44ms/step - loss: 0.5993 - acc: 0.8130 - val_loss: 0.7609 - val_acc: 0.7729
Epoch 7/30
781/781 [==============================] - 32s 41ms/step - loss: 0.5905 - acc: 0.8174 - val_loss: 0.7538 - val_acc: 0.7706
Epoch 8/30
781/781 [==============================] - 32s 42ms

In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(quantized_model) # path to the SavedModel directory

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save the model.
with open(os.path.join(base_path, 'model_quant.tflite'), 'wb') as f:
  f.write(tflite_model)

/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [11]:
print("Float model in Kb:", os.path.getsize(os.path.join(base_path, "model.tflite")) / float(2**10))
print("Quantized model in Kb:",  os.path.getsize(os.path.join(base_path, "model_quant.tflite")) / float(2**10))


Float model in Kb: 290.26171875
Quantized model in Kb: 183.390625
